In [2]:
import re
import pymongo
import datetime
import pandas as pd
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [34]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)
base_url = "https://beta.bls.gov/dataQuery/"
starting_url = base_url + "find?q=data+scientists&r=100&st=0"
browser.visit(starting_url)
search_html = browser.html
soup_search = bs(search_html, 'html.parser')
data_query_results = soup_search.find_all("div", class_='dq-result-item')

url_dict = {"linkID":[],"description":[],"url":[]}
data_query_df = pd.DataFrame([])

last_link_on_page = data_query_results[-1].a.get('href')

text_parent = soup_search.find_all("div", class_='dq-page-links')
pages = text_parent[0].span.text.split()[-1]

# want to put number of pages into loop boundary range(pages)
for page in range(2):
    search_html = browser.html
    soup_search = bs(search_html, 'html.parser')
    data_query_results = soup_search.find_all("div", class_='dq-result-item')
    for result in data_query_results:
        result_url = result.a.get('href')
        link_title = result.label.get_text()
        link_id = result.input.get('value')
        # print(link_title)
        # print(link_id)
        if result_url != last_link_on_page:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
        else:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
            next_div = soup_search.find_all("div", class_='dq-next-page')
            next_url = base_url + next_div[0].a.get('href')
            print(next_url)
    browser.visit(next_url)

url_dict_df = pd.DataFrame(url_dict)

url_dict_df

# <div id="dq-num-results-wrapper">
#            Showing Results
#            <strong>
#             1
#                             -
#                             100
#            </strong>
#            of
#            <strong>
#             5358
#            </strong>
#           </div>





====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/justin/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
https://beta.bls.gov/dataQuery/find?st=100&r=100&q=data+scientists&more=0


,linkID,description,url
0,OEUN000000051800015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
1,OEUN000000051820015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
2,OEUN000000051700015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
3,OEUN000000051800015209803,Hourly mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
4,OEUN000000051800015209804,Annual mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
...,...,...,...
195,OEUN000000061100315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
196,OEUN000000061130015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
197,OEUN000000061120215209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
198,OEUN000000061120315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...


In [20]:
browser.quit()

In [27]:
text_parent = soup_search.find_all("div", class_='dq-page-links')
last_page_number = int(text_parent[0].span.text.split()[-1])
print(last_page_number)

54


In [43]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

# seriesDataTable1

for url in range(len(url_dict["url"])):
    print(url_dict["url"][url])



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/justin/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051700015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209808
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209813
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209808
https://beta.bls.gov/dataViewer/view/timeserie

In [15]:
bls_state_data_df = pd.read_csv('state_M2020_dl.csv')
state_df = bls_state_data_df.loc[bls_state_data_df["AREA_TITLE"]=="Alabama", :]
job_in_state_df = state_df.loc[state_df["OCC_CODE"]=="15-2098", :]
job_in_state_df 

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
77,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,15-2098,Data Scientists and Mathematical Science Occup...,...,43.55,50.79,58.59,"56,510","74,460","90,590","105,650","121,870",NaN,NaN
